# Basic model construction

## Objective
In this notebook, we will use the `summer` interface 
to demonstrate the construction of a simple model,
and illustrate several principles of infectious disease modelling.
Because this is the first practical notebook/chapter, 
there is slightly more code-related content in this chapter.
We'll come back to some of the features of our simple model in the following notebooks,
including thinking more about flows and how we obtain numerical solutions for our system.
In our example model, we will create an SIR compartmental model for an unspecified, short-lived, immunising infectious disease spreading through a fully susceptible population.

In this model there will be:

- Three compartments, named `susceptible`, `infectious` and `recovered`
- A starting population of 1000 people, with 10 of them infected (and infectious)
- An evaluation timespan from time zero to 20 units of time
(which we can think of as days, even though this doesn't need to be stipulated)
- Inter-compartmental flows for the infection and recovery processes

Susceptible, infectious, recovered is a very commonly used model structure,
although many variations on this exist,
some of which will be explored in later notebooks.
These compartments are commonly abbreviated to "S", "I" and "R",
a convention which we will use in our diagrams.
The recovered compartment is also commonly termed
the "removed" compartment, because the population in this compartment
have no effect on transmission
(except potentially by contributing to the total population size).

## Preliminaries
As with any Python script, we first have to import the objects we need,
plus we'll set the visualisation of pandas objects to be interactive.
You'll see similar cells to these in all the notebooks in the series.

In [ ]:
try:
    import google.colab
    %pip install summerepi
except:
    pass

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"

from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput

## Building a simple SIR model with summer
First, let's create a function that gives us a base SIR model with the
basic compartments, starting populations and inter-compartmental flows implemented.
The following is a diagram representing this.
Diagrams such as these are a common way to represent compartmental models,
with the compartments represented as boxes and the flows as arrows.
Such diagrams give a quick visual impression of the system
and may be more intuitive than the `summer` code that actually
constitutes the system or ODEs intended to represent it.
However, unlike code and ODEs, there is no formal guide to specify
exactly how these diagrams should be constructed and so 
they should not be considered as the definitive representation of the system.
If in doubt, always look at the actual model code itself.

![](../images/sir_structure.svg)

Now let's build this model using `summer`. 
Note the steps in model construction described in the comments
interspersed through the following function.

In [ ]:
# First, initialise a basic summer compartmental model,
# specifying the analysis times, compartment names and the name of infectious compartment
sir_model = CompartmentalModel(
    times=(0.0, 20.0),
    compartments=(
        "susceptible",
        "infectious",
        "recovered",
    ),
    infectious_compartments=("infectious",),
)

# Assign the starting population
sir_model.set_initial_population(
    distribution=
    {
        "susceptible": 990.0, 
        "infectious": 10.0,
    },
)

# Add a dynamic infection flow that transitions people from susceptible to infectious
sir_model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=1.0,
    source="susceptible", 
    dest="infectious",
)

# Add a constant flow that transitions people from infectious to recovered
sir_model.add_transition_flow(
    name="recovery", 
    fractional_rate=0.333,
    source="infectious", 
    dest="recovered",
)

# Run the model
sir_model.run()

# Display the outputs
# Note that we previously set the pandas plotting backend to plotly for interactive plots
sir_model.get_outputs_df().plot(
    labels={"index": "time", "value": "compartment size"},
)

So we already have a basic model set up in only a few commands
that also express the code's intention.
## The code style we will adopt from here onward
There are a few aspects of this Python code that can be improved and
will make life easier for ourselves later on in the series.
Throughout the remaining notebooks, 
we'll adhere to these "better" standards of programming, 
which should make the code more expressive,
improve the notebook interface and allow us to write more efficient code
(e.g. when we come to [calibrating our models](./20-calibration.ipynb).
Specifically, we are going to:
- Package up the construction of our model into a function
- Explicitly declare the "type" of the data structures that
are expected to go into this function and to come back out at the end
- Include a "docstring" (multi-line comment)
for the function to explain the function's purpose in more detail
- Distinguish numerical parameters that we might like to adjust later from 
general settings or instructions on how the model should be build (model "configurations")
- Use _summer2_ `Parameter` objects rather than `float`s for our model parameters

In [ ]:
def build_sir_model(  # Declare a Python function
    config: dict,  # ... that expects a dictionary as the type of it's one argument
) -> CompartmentalModel:  # ... and returns a summer compartmental model object
    """
    This is the function docstring.

    Generate an instance of an SIR model from a set of requested configurations/settings,
    with a fixed set of compartment names, a fixed infectious compartment
    and a fixed set of flows between the compartments.
    
    Args:
        config: Quantities needed for model construction (other than model parameters)
    Returns:
        model: The summer model object
    """

    compartments = (
        "susceptible",
        "infectious",
        "recovered",
    )
    analysis_times = (0.0, config["end_time"])
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=("infectious",),
    )
    model.set_initial_population(
        distribution=
        {
            "susceptible": config["population"] - config["seed"], 
            "infectious": config["seed"],
        },
    )
    
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("contact_rate"),
        source="susceptible", 
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=Parameter("recovery"),
        source="infectious", 
        dest="recovered",
    )
    
    return model

## Getting the model, running it and examining the results
Now we can use our model building function to get an instance of this model,
run it, and have a look at the compartment size progression over time.
Note that we use the plotting functions that are built into `pandas` objects to do this.
`Pandas` is a very widely used library for data processing, which we will use extensively in this series.
Because there may be considerable output wrangling necessary 
after we have run our model and this wrangling is not infectious diseases-specific,
we'll use external libraries such as `pandas` for these steps.

Next, decide on some settings to use to set up the model object.

In [ ]:
# Declare some quantities that we don't intend to change
model_config = {
    "population": 1000.0,
    "seed": 10.0,
    "end_time": 20.0,
}

# Build the model with this configuration
sir_model = build_sir_model(model_config)

Now we have a model object ready to run,
except that we need to specify what our parameter values are.
This will stand us in good stead as our models get more complicated,
although there are few strict rules here.
For example, we could have chosen to make "population" a parameter
rather than part of `config`.
Perhaps most importantly,
it's better not to have to build a new model
every time we want to run it with new parameters
(although it doesn't make much difference here).

In [ ]:
# Now specify all the rate parameters we need
parameters = {
    "recovery": 0.333,
    "contact_rate": 1.0,
}

# Run with these parameters and plot the outputs
sir_model.run(parameters=parameters)
compartment_values = sir_model.get_outputs_df()
compartment_values.plot(
    labels={"index": "time", "value": "compartment size"},
)

This is the point to start playing around with the code!
Whether you're accessing this over Google's Colab interface or locally as a downloaded notebook,
we encourage you to make changes to the previous two cells.
Hopefully it's apparent that this is easier now that everything is packaged up 
in the way we have shown in the previous two cells.
That is, you can adjust parameters and immediately re-run the model in the previous cell
without needing to re-build the model each time you run.
If you do need to re-build the model with the same compartmental structure
but with some different specifications about how you'll run the model,
you can adjust the cell before and then re-run with the parameters you want to apply in the last cell.
(i.e. the cell containing the `build_sir_model` function).
We'll see later how this will make processes like calibration more efficient.
## Digging into the model object
Now that we have our `CompartmentalModel` object,
we can use this structure to inspect some aspects of what is going on under the surface;
for example, compartments, flows and other attributes.
This is **_highly recommended_**, 
to ensure that the model you have created is consistent with what you were expecting.
Try out using tab complete in this notebook to inspect the range of methods and
attributes that are available for a `CompartmentalModel` object.

In [ ]:
print(f"Modelled compartments are: {sir_model.compartments}")
print("\nFlows implemented are:")
for i_flow in sir_model.flows:
    print(i_flow)
print(f"\nEvaluation times are: {sir_model.times}")
print("\nResulting compartment sizes over time are:")
compartment_values

So it looks like we have a model object that is consistent with our intentions.

## Epidemiological messages
This is clearly a very simple model of an epidemic caused by a short-lived pathogen that induces complete immunity in its host.
However, despite its simplicity, it does capture a surprising number of the actual features of an epidemic caused by an infection
of this type.
In general, models of infectious diseases transmission should be as complicated as they need to be,
which means that the additional complexity that we might need to inject into this model is highly dependent on the purpose that
we will be using it for - or the epidemiological question that we will be addressing through our analysis.
It may also be dependent on us having sufficient epidemiological understanding of the epidemic to be able to incorporate these
features - with a reasonable level of confidence that we are actually capturing the processes that we are interested in
(including empiric data being available to estimate the parameters that we need to build our more complicated model).

Let's think of some of the epidemiological features that this very simple model **_does_** capture:
- Very broadly, this model gives us the shape that epi curves often follow - looking vaguely like a bell
- There is an exponential growth phase when the population remains largely susceptible
- Transmission declines as the proportion of the population that is susceptible decreases
- When the effective reproduction number falls below one, the epidemic peaks and begins to decline
- As the epidemic ends and transmission declines towards zero, susceptibles are depleted, but not completely - a proportion of the population remains susceptible even after the epidemic

Let's also think of some of the epidemiological features that this model **_does not_** capture:
- Any heterogeneity in the background population with regards progression through infection states after exposure
- Any heterogeneity in transmission, such as greater transmission between people within the population with similar characteristics
- Any heterogeneity in the pathogen, such as multiple strains with different characteristics circulating through the population
- Any changes in how people transition through their stages over time that might be induced through changes external to the model
(i.e. other than those related to the changes in the population distribution across compartments resulting from transmission of this immunising pathogen)
- Tracking any modelled quantities other than the sizes of the model's compartments

We will return to these features and how to elaborate our base model to capture them over the following notebooks.

### Illustrations
Note the approximately straight line at the start of the epidemic when the vertical axis is displayed on a log scale,
implying exponential growth.
During these first three or four time units,
the susceptible population remains approximately unchanged,
and so the epidemic expands according to the reproduction number (see [notebook 10](./10-reproduction-number.ipynb)).

In [ ]:
px.line(
    x=compartment_values.index, 
    y=compartment_values["infectious"], 
    labels={"x":"time", "y":"infectious"}, 
    log_y=True,
)

Last, let's illustrate that the number of infectious persons plateaus 
and begins to fall at the point that the reproduction number declines below one.
We'll need to keep track of a few quantities in the model to show this.
The number of persons infected by an infectious person in a fully susceptible
population (the base reproduction number or $R_{0}$)
can be intuitively calculated as the number of persons infected by
an infectious individuals per unit time (i.e. the `contact_rate` parameter)
multiplied by the length of time they spend infectious 
(i.e. the reciprocal of the `recovery` parameter).
This base reproduction number can be multipled 
by the proportion of the population that remains susceptible
to calculate the net reproduction number (often termed $R_{n}$).
By keeping track of this quantity, we can demonstrate that the 
infectious population will begin to fall when this quantity falls below one.
(This is most easily done by tracking quantities that emerge
from the model during the integration process,
which is described in more detail in [notebook 08](./08-derived-outputs.ipynb).

In [ ]:
def get_sir_model_with_reproduction_number(
    config: dict,
) -> CompartmentalModel:
    """
    Take the previous build function to build an SIR model
    and track enough quantities to calculate the effective reproduction number.
    """
    model = build_sir_model(config)

    model.request_output_for_compartments(
        "n_susceptible",
        ("susceptible",),
        save_results=False,
    )
    model.request_output_for_compartments(
        "total_population",
        sir_model.compartments,
        save_results=False,
    )
    model.request_function_output(
        "prop_susceptible",
        func=DerivedOutput("n_susceptible") / DerivedOutput("total_population"),
        save_results=False,
    )
    model.request_function_output(
        "net_reproduction_number",
        func=DerivedOutput("prop_susceptible") * Parameter("contact_rate") / Parameter("recovery"),
    )
    
    return model

In [ ]:
sir_model_with_rn = get_sir_model_with_reproduction_number(model_config)
sir_model_with_rn.run(parameters=parameters)

In [ ]:
times = sir_model_with_rn.times

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(
        x=times, 
        y=sir_model_with_rn.get_derived_outputs_df()["net_reproduction_number"], 
        name="reproduction number value",
    ),
)
fig.add_trace(
    go.Scatter(
        x=times,
        y=[1.0] * len(times),
        name="reproduction number threshold",
    ),
)
fig.add_trace(
    go.Scatter(
        x=times, 
        y=sir_model_with_rn.get_outputs_df()["infectious"], 
        name="infectious population",
    ),
    secondary_y=True,
)
fig.update_xaxes(title_text="time")
fig.update_yaxes(title_text="net reproduction number")
fig.update_yaxes(title_text="number infectious", secondary_y=True)
fig.show()

So already we have been able to demonstrate 
some important epidemiological principles even with a very simple model.
We'll get into several of these principles in more detail
in the following notebooks,
and also come back to several of the process and commands
we used in constructing the model.